# Analyse 1 : utilisation de regexp et de patterns spacy

on essaie d'extraire des termes connus, et des locutions typiques, pour détecter les références aux personnages et aux lieux.



## Trouver et lister les fichiers

In [7]:
from glob import glob
import os.path

In [ ]:
# FIXME fournir code lecture fichiers depuis Google Drive ici (archive format zip)

In [8]:
PATH_TEXT_FILES_DIR = "/home/joseph/tmp/French_ELTEC_NER_Open_Dataset/texts"

In [10]:
files = sorted(glob(os.path.join(PATH_TEXT_FILES_DIR, "*.txt")))
print("Found", len(files), "files.")
files[:10]

Found 100 files.


['/home/joseph/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA00101_Adam.txt',
 '/home/joseph/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA00102_Adam.txt',
 '/home/joseph/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA00201_Audoux.txt',
 '/home/joseph/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA00301_Aimard.txt',
 '/home/joseph/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA00302_Aimard.txt',
 '/home/joseph/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA00401_Allais.txt',
 '/home/joseph/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA00501_Balzac.txt',
 '/home/joseph/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA00502_Balzac.txt',
 '/home/joseph/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA00503_Balzac.txt',
 '/home/joseph/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA00601_Boisgobey.txt']

In [11]:
sample = files[50]
sample

'/home/joseph/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA03001_Ohnet.txt'

## Import spacy and start processing

In [ ]:
import spacy


## L'objet `nlp`
On construit une nouvelle chaîne de traitements de plusieurs façon. La manière la plus simple est de construire une chaîne de traitement vide (ou presque) pour le français à l'aide de la "fabrique" à chaînes de traitement `spacy.blank(LANGAGE)`.

In [ ]:
nlp = spacy.blank("fr")


La chaîne de traitements contient différents traitements appliqués les uns après les autres.
On peut afficher cette liste de traitements à l'aide de l'attribut `pipe_names` de l'objet `nlp`.

In [13]:
nlp.pipe_names

[]

Par défaut, une chaîne de traitement ne contient rien… Sauf un *tokenizer*, d'où l'importance de préciser la langue !

## L'objet `doc`
On obtient un objet `doc` en appliquant la chaîne de traitement `nlp` à une chaîne de texte.


In [23]:
# Créé en traitant une chaine de caractères avec l'objet nlp
doc = nlp("Bonjour tout le monde !")

Cet objet `doc` est central pour Spacy : il contient toute les informations produite par la chaîne de traitement à propos de notre texte.

On peut parcourir les *tokens* extraits à l'aide d'une boucle classique en Python.

In [24]:
# Itère sur les tokens dans un Doc
for token in doc:
    print(token.text)

Bonjour
tout
le
monde
!


In [25]:
# On peut sélectionner un token particulier, grâce à son indice dans le document
token = doc[1]
token

tout

### Exemple de cellule avec contenu caché par défaut
#### 🚧 <b>Essayez à présent de sélectionner et afficher les <i>tokens</i> "tout le monde".</b>

<details>
<summary>Indices</summary>

Vous pouvez utiliser les *ranges* pour sélectionner plusieurs éléments d'un itérable. Voici un exemple de la syntaxe à utiliser :
```python
ma_liste = [0, 1, 2, 3]
print(ma_liste[1:3])
```
</details>

<details>
<summary>Solution</summary>

On applique cette syntaxe pour sélectionner les tokens du rang 1 (2e token, inclus) au rang 4 (non inclus) :

```python
doc[1:4]
```
</details>


In [29]:
# On peut également utiliser les "ranges" Python pour sélectionner plusieurs tokens
span = doc[1:4]
span

tout le monde

In [30]:
# On peut également accéder aux attributs d'un span
span.text

'tout le monde'

Lorsqu'on souhaite traiter plusieurs documents, on peut utiliser `nlp.pipe(LISTE_DE_TEXTES)`.
Dans ce cas, on obtient une liste de documents en sortie, qu'il est possible d'inspecter avec une seconde boucle !

In [31]:
TEXTES = [
    "Bonjour tout le monde !", 
    "Comment allez-vous ? Bien, j'espère !",
    "Savez-vous qu'une chaîne de caractères peut contenir des retours à la ligne\ncomme celui-ci ?"
    ]
documents = nlp.pipe(TEXTES)
for doc_id, doc in enumerate(documents):
    for token in doc:
        print(f"doc#{doc_id}, tok#{token.i}: {token.text}")

doc#0, tok#0: Bonjour
doc#0, tok#1: tout
doc#0, tok#2: le
doc#0, tok#3: monde
doc#0, tok#4: !
doc#1, tok#0: Comment
doc#1, tok#1: allez
doc#1, tok#2: -vous
doc#1, tok#3: ?
doc#1, tok#4: Bien
doc#1, tok#5: ,
doc#1, tok#6: j'
doc#1, tok#7: espère
doc#1, tok#8: !
doc#2, tok#0: Savez
doc#2, tok#1: -vous
doc#2, tok#2: qu'
doc#2, tok#3: une
doc#2, tok#4: chaîne
doc#2, tok#5: de
doc#2, tok#6: caractères
doc#2, tok#7: peut
doc#2, tok#8: contenir
doc#2, tok#9: des
doc#2, tok#10: retours
doc#2, tok#11: à
doc#2, tok#12: la
doc#2, tok#13: ligne
doc#2, tok#14: 

doc#2, tok#15: comme
doc#2, tok#16: celui-ci
doc#2, tok#17: ?


## 

In [32]:
## Autres attributs des tokens et des spans

doc = nlp("Cela coûte 5 €.")

print("Index :   ", [token.i for token in doc])
print("Text :    ", [token.text for token in doc])

print("is_alpha :", [token.is_alpha for token in doc])
print("is_punct :", [token.is_punct for token in doc])
print("like_num :", [token.like_num for token in doc])

Index :    [0, 1, 2, 3, 4]
Text :     ['Cela', 'coûte', '5', '€', '.']
is_alpha : [True, True, False, False, False]
is_punct : [False, False, False, False, True]
like_num : [False, False, True, False, False]


In [66]:

# Process a text
doc = nlp("La journée de formation à Lyon se déroule bien.")
for token in doc:
    # print(token.text, token.pos_, token.dep_, token.head.text)
    print(f"{token.text:>10s}", f"{token.pos_:>6s}", f"{token.dep_:>12s}", f"{token.head.text:>10s}")

        La    DET          det    journée
   journée   NOUN        nsubj    déroule
        de    ADP         case  formation
 formation   NOUN         nmod    journée
         à    ADP         case       Lyon
      Lyon  PROPN         nmod    journée
        se   PRON    expl:comp    déroule
   déroule   VERB         ROOT    déroule
      bien    ADV       advmod    déroule
         .  PUNCT        punct    déroule


In [67]:
from spacy import displacy

In [68]:
displacy.render(doc, style="ent", jupyter=True)

## Utilisation d'un pipeline avec un pos_tagger et un reconnaisseur d'entités nommées

In [35]:
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 25.5 MB/s eta 0:00:00m eta 0:00:010:01:01
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


In [36]:

nlp = spacy.load("fr_core_news_sm")

In [49]:
def load_text(filename: str) -> str:
    with open(filename, encoding="utf8") as in_file:
        return "".join(in_file.readlines())

In [55]:
text = load_text(files[50])
print(text)
doc = nlp(text)

I
Dans un de ces charmants chemins creux de Normandie, serpentant entre les levées, plantées de grands arbres, qui entourent les fermes d'un rempart de verdure impénétrable au vent et au soleil, par une belle matinée d'été, une amazone, montée sur une jument de forme assez médiocre, s'avançait au pas, les rênes abandonnées, rêveuse, respirant l'air tiède, embaumé du parfum des trèfles en fleurs. Avec son chapeau de feutre noir entouré d'un voile de gaze blanche, son costume de drap gris fer à longue jupe, elle avait fière tournure. On eût dit une de ces aventureuses grandes dames qui, au temps de Stofflet et de Cathelineau, suivaient hardiment l'armée royaliste, dans les traînes du Bocage, et éclairaient de leur sourire la sombre épopée vendéenne.
Élégante et svelte, elle se laissait aller gracieusement au mouvement de sa monture, fouettant distraitement de sa cravache les tiges vertes des genêts. Un lévrier d'Écosse au poil rude et rougeâtre l'accompagnait, réglant son allure souple s

In [59]:
# Itère sur les entités prédites
for ent in doc.ents:
    # Affiche le texte de l'entité et son label
    print(ent.text, ent.label_)

I LOC
Normandie LOC
Stofflet PER
Cathelineau LOC
Bocage LOC
Élégante ORG
Écosse LOC
amazone LOC
Madame PER
Rassurez PER
Monsieur PER
Allons PER
Fox ORG
Où MISC
La Neuville LOC
Madame PER


In [63]:
spacy.explain("GPE")

'Countries, cities, states'

In [64]:
spacy.explain("LOC")

'Non-GPE locations, mountain ranges, bodies of water'

## Utilisation du matcher Spacy
TODO montrer comment filtrer sur lexème, nature ou fonction du token.

In [58]:
pattern = [{"POS": "PROPN"}]

In [ ]:
spacy.explain("PROPN")

'proper noun'

In [56]:
from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)
matcher.add("NOMS_PROPRES", [pattern])
matches = matcher(doc)

In [57]:
print("Nombre de correspondances trouvées :", len(matches))

# Itère sur les correspondances et affiche la portion de texte
for match_id, start, end in matches:
    print("Correspondance trouvée :", doc[start:end].text)

Nombre de correspondances trouvées : 10
Correspondance trouvée : Normandie
Correspondance trouvée : Stofflet
Correspondance trouvée : Cathelineau
Correspondance trouvée : Écosse
Correspondance trouvée : Rassurez
Correspondance trouvée : Allons
Correspondance trouvée : Fox
Correspondance trouvée : Neuville
Correspondance trouvée : volontiers
Correspondance trouvée : hasard


In [ ]:
## Ouverture : utilisation de la bibliothèque Transformers

# https://huggingface.co/Jean-Baptiste/camembert-ner (même API !)

from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner")
model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner")


##### Process text sample (from wikipedia)

from transformers import pipeline

nlp = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="simple")
nlp("Apple est créée le 1er avril 1976 dans le garage de la maison d'enfance de Steve Jobs à Los Altos en Californie par Steve Jobs, Steve Wozniak et Ronald Wayne14, puis constituée sous forme de société le 3 janvier 1977 à l'origine sous le nom d'Apple Computer, mais pour ses 30 ans et pour refléter la diversification de ses produits, le mot « computer » est retiré le 9 janvier 2015.")
